## Overview
When you train a Gaia-only model of stellar magnitude, a couple of obvious artifacts show up in a chart of residual outlier positions, at opposite ends of the celestial sphere. A list of the problematic Gaia DR2 source_id's from the [input dataset](https://www.kaggle.com/solorzano/257k-gaia-dr2-stars) is made available in the output tab.

## Data
We will use a [dataset of 257K Gaia DR2 stars](https://www.kaggle.com/solorzano/257k-gaia-dr2-stars) in the Northern and Southern Hemispheres, with magnitude of at most 13.5 and parallax of at least 2.0 mas (500 parsecs).

In [ ]:
import pandas as pd

data = pd.read_csv('../input/257k-gaiadr2-sources-with-photometry.csv', dtype={'source_id': str})

In [ ]:
len(data)

## Removal of duplicates
The dataset contains row pairs that have the same *source_id*. It appears that some Gaia DR2 sources match two Tycho2 sources. We'll simply remove the problematic pairs before proceeding.

In [ ]:
_seen = set()
should_remove_set = set()
for _source_id in data['source_id']:
    if _source_id in _seen:
        should_remove_set.add(_source_id)
    else:
        _seen.add(_source_id)

In [ ]:
data = data[~data['source_id'].isin(should_remove_set)].reset_index(drop=True)
len(data)

In [ ]:
assert len(data) == len(set(data['source_id']))

## Modeling helper functions
This function is just boilerplate:

In [ ]:
import inspect

pd_concat_argspec = inspect.getfullargspec(pd.concat)
pd_concat_has_sort = 'sort' in pd_concat_argspec.args

def pd_concat(frames):
    # Due to Pandas versioning issue
    new_frame = pd.concat(frames, sort=False) if pd_concat_has_sort else pd.concat(frames)
    new_frame.reset_index(inplace=True, drop=True)
    return new_frame

The following function is used to train a regression model, averaging multiple runs of k-fold cross-validation. It can use an arbitrary sklearn-style regressor (one with a *fit* function.) 

In [ ]:
import types
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler 

np.random.seed(201808011)

def model_results(data_frame, label_extractor, var_extractor, trainer_factory, id_column='source_id', n_splits=2, n_runs=3, scale=False, max_n_training=None):
    '''
    Returns a frame with source_id, response and residual columns, with the same ordering as data_frame.
    '''
    sum_series = pd.Series([0] * len(data_frame))
    for r in range(n_runs):
        shuffled_frame = data_frame.sample(frac=1)
        shuffled_frame.reset_index(inplace=True, drop=True)
        response_frame = pd.DataFrame(columns=[id_column, 'response'])
        kf = KFold(n_splits=n_splits)
        for train_idx, test_idx in kf.split(shuffled_frame):
            train_frame = shuffled_frame.iloc[train_idx]
            if max_n_training is not None:
                train_frame = train_frame.sample(max_n_training)
            test_frame = shuffled_frame.iloc[test_idx]
            train_labels = label_extractor(train_frame) if isinstance(label_extractor, types.FunctionType) else train_frame[label_extractor]
            train_vars = var_extractor(train_frame)
            test_vars = var_extractor(test_frame)
            if scale:
                scaler = StandardScaler()  
                scaler.fit(train_vars)
                train_vars = scaler.transform(train_vars)  
                test_vars = scaler.transform(test_vars) 
            trainer = trainer_factory()
            fold_model = trainer.fit(train_vars, train_labels)
            test_responses = fold_model.predict(test_vars)
            test_id = test_frame[id_column]
            assert len(test_id) == len(test_responses)
            fold_frame = pd.DataFrame({id_column: test_id, 'response': test_responses})
            response_frame = pd_concat([response_frame, fold_frame])
        response_frame.sort_values(id_column, inplace=True)
        response_frame.reset_index(inplace=True, drop=True)
        assert len(response_frame) == len(data_frame), 'len(response_frame)=%d' % len(response_frame)
        sum_series += response_frame['response']
    cv_response = sum_series / n_runs
    assert len(cv_response) == len(data_frame)
    sorted_result = pd.DataFrame({
        id_column: np.sort(data_frame[id_column].values), 
        'response': cv_response})
    data_frame_partial = pd.DataFrame({id_column: data_frame[id_column]})
    merged_frame = pd.merge(data_frame_partial, sorted_result, how='inner', on=id_column, sort=False)
    data_frame_labels = label_extractor(data_frame) if isinstance(label_extractor, types.FunctionType) else data_frame[label_extractor]
    merged_frame['residual'] = data_frame_labels - merged_frame['response']
    assert len(merged_frame) == len(data_frame)
    return merged_frame

We will also define a function that lets us evaluate the responses of a model trained with the *model_results* function.

In [ ]:
import math
import scipy.stats as stats

def print_evaluation(data_frame, label_column, response_frame):
    _response = response_frame['response']
    _label = label_column(data_frame) if isinstance(label_column, types.FunctionType) else data_frame[label_column]
    _error = _label - _response
    assert sum(response_frame['residual'] == _error) == len(data_frame)
    _rmse = math.sqrt(np.sum(_error ** 2) / len(data_frame))
    _correl = stats.pearsonr(_response, _label)[0]
    print('RMSE: %.4f | Correlation: %.4f' % (_rmse, _correl,), flush=True)

## Variables
We'll keep it simple: Model variables will only include distance, logarithm of distance, and a couple "color index" features from Gaia itself. We're neglecting extinction near the galactic plane, but that's OK for purposes of this kernel.

In [ ]:
def extract_mag_model_vars(data_frame):
    distance = (1000.0 / data_frame['parallax']).values
    log_distance = np.log(distance)

    feature_list = [log_distance, distance]
    feature_list.append(data_frame['phot_g_mean_mag'] - data_frame['phot_rp_mean_mag'])
    feature_list.append(data_frame['phot_bp_mean_mag'] - data_frame['phot_g_mean_mag'])
    
    return np.transpose(feature_list)    

## Magnitude model
The regression label will be the *g* magnitude column. We'll train the model with a Neural Network. A GBM, for example, would also do, but it seems that Neural Network results are clearer in the outlier visualization.

In [ ]:
LABEL_COLUMN = 'phot_g_mean_mag'
MAX_N_TRAINING = 40000

In [ ]:
from sklearn.neural_network import MLPRegressor

def get_mag_trainer():
    return MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=500, alpha=0.1, random_state=np.random.randint(1, 10000))

In [ ]:
mag_results = model_results(data, LABEL_COLUMN, extract_mag_model_vars, get_mag_trainer, n_runs=2, scale=True, max_n_training=MAX_N_TRAINING)

In [ ]:
print_evaluation(data, LABEL_COLUMN, mag_results)

## Outlier positions
Let's get 1000 outliers from each end of the model's residual distribution, and present them in positional scatter charts.

In [ ]:
NUM_OUTLIERS = 1000

In [ ]:
def produce_outliers():
    global data_dim_outliers
    global data_bright_outliers
    global idx_dim_outlier
    global idx_bright_outlier
    
    idx_res_sort = np.argsort(mag_results['residual'].values)
    idx_dim_outlier = idx_res_sort[-NUM_OUTLIERS:]
    idx_bright_outlier = idx_res_sort[:NUM_OUTLIERS]
    data_dim_outliers = data.iloc[idx_dim_outlier]
    data_bright_outliers = data.iloc[idx_bright_outlier]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_outliers():
    plt.rcParams['figure.figsize'] = (14, 7)
    plt.scatter(data_dim_outliers['ra'], data_dim_outliers['dec'], color='blue', s=8)
    plt.scatter(data_bright_outliers['ra'], data_bright_outliers['dec'], color='red', s=8)
    plt.title('Dim (blue) and bright (red) outliers')
    plt.xlabel('Right Ascension (degrees)')
    plt.ylabel('Declination (degrees)')
    plt.show()
    plt.scatter(data_dim_outliers['l'], data_dim_outliers['b'], color='blue', s=8)
    plt.scatter(data_bright_outliers['l'], data_bright_outliers['b'], color='red', s=8)
    plt.title('Dim (blue) and bright (red) outliers')
    plt.xlabel('Galactic longitude (degrees)')
    plt.ylabel('Galactic latitude (degrees)')
    plt.show()
    
produce_outliers()    
plot_outliers()

## Removal of stars with apparent systematic issue
There is a blue horizontal line of abnormally dim stars at a declination of about -65°, with a right ascension near 90°. A very similar feature also appears at the exact opposite end of the celestial sphere.

It's not clear what causes this artifact. Removal will be ad-hoc. The following bounding box covers the artifact in the bottom-left of the RA-DEC chart.

In [ ]:
S_RA_MIN = 65
S_RA_MAX = 115
S_DEC_MIN = -68
S_DEC_MAX = -64

Let's define a boolean series (*s_mask*) that is *True* only for *data* rows that are either in the bounding box defined above, or an equivalent box at the opposite end of the celestial sphere.

In [ ]:
data_ra = data['ra']
data_dec = data['dec']
s_mask_bottom_left = (data_ra >= S_RA_MIN) & (data_ra <= S_RA_MAX) & (data_dec >= S_DEC_MIN) & (data_dec <= S_DEC_MAX)
s_mask_top_right = (data_ra >= S_RA_MIN + 180) & (data_ra <= S_RA_MAX + 180) & (data_dec >= -S_DEC_MAX) & (data_dec <= -S_DEC_MIN)
s_mask = s_mask_bottom_left | s_mask_top_right

Let's also make sure we capture this set of stars in the global collection we will write to the output tab.

In [ ]:
should_remove_set = should_remove_set.union(set(data[s_mask]['source_id']))

In [ ]:
len(should_remove_set)

Now we can remove the problematic sources from the *data* frame and the frame containing the results of the regression model.

In [ ]:
data = data[~s_mask]
data.reset_index(inplace=True, drop=True)
mag_results = mag_results[~s_mask]
mag_results.reset_index(inplace=True, drop=True)

In [ ]:
len(data)

In [ ]:
assert np.sum(mag_results['source_id'] == data['source_id']) == len(data)

## New charts
What do outlier visualizations look like now?

In [ ]:
produce_outliers()
plot_outliers()

Seems alright.

## Output
Let's dump the list of source_id's that should be removed from the original dataset. Note that they not only include sources with the positional systematic involving anomalously dim stars, but also those that have multiple Tycho2 matching sources.

In [ ]:
should_remove_frame = pd.DataFrame({
    'source_id': list(should_remove_set)
})

In [ ]:
should_remove_frame.to_csv('257k-gaiadr2-should-remove.csv', index=False)

## Acknowledgments

This work has made use of data from the European Space Agency (ESA) mission Gaia (https://www.cosmos.esa.int/gaia), processed by the Gaia Data Processing and Analysis Consortium (DPAC, https://www.cosmos.esa.int/web/gaia/dpac/consortium). Funding for the DPAC has been provided by national institutions, in particular the institutions participating in the Gaia Multilateral Agreement.